In [8]:
# Import package
import os
import glob as glob
import matplotlib.pyplot as plt
import random
import cv2
import os
import numpy as np
from pathlib import Path
import xml.dom
from xml.dom.minidom import parse
from shutil import copyfile

In [2]:
#!git clone https://github.com/meituan/YOLOv6.git

Cloning into 'YOLOv6'...


In [19]:
# DEFINE PATHS
file_root = ""
imgset_root = "BCCD/ImageSets/Main"
img_path = "BCCD/JPEGImages"
annotation_path = "BCCD/Annotations"
data_root = ""
label_root = "BCCD/Labels"
des_imgs_path = "images"
des_labels_path = "labels"

In [20]:
# DEFINE CLASS LABEL
classes = ['Platelets', 'RBC', 'WBC']

In [21]:
def cord_converted(size, box):
    """
    convert xml annotation (COCO) to darknet format coordinates (YOLO)
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])  
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    #Calculate relative rate between x,y and size_x size_y to convert into relative cordinate range(0,1]

    dw = np.float32(1. / int(size[0])) 
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

In [22]:
def save_file(jpg_file_name, size, img_box):
    """
    Convert infor extract from get_data_xml to annotation txt file
    Input:
    - jpg_file_name: jpg img dir
    - img_box: COCO annotation ready to convert
    - size: jpg_file_name size

    :return: txt annotation file
    """

    save_file_name = label_root + '/' + jpg_file_name + '.txt'
  # Open txt annotation file from dir with write append mode
    file_path = open(save_file_name, "a+")
    for box in img_box:
      cls_num = classes.index(box[0]) #Get index of label in annotation file
      new_box = cord_converted(size, box[1:]) #Convert to YOLO format
      # Write new_annotation to txt annotation file
      file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")
    file_path.flush()
    file_path.close()

In [23]:
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    #print(img_path)

    dom = parse(img_path)  #read xml file
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data #get file name from xml file
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:  #Consider everybox bbox in detected objects in xml annotation
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        # print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2]) # Add COCO annotation ready for converting
    save_file(img_xml_file, [img_w, img_h], img_box)

In [24]:
files = os.listdir(annotation_path)
for file in files:
  print("file name: ", file)
  file_xml = file.split(".") # Cut .xml
  get_xml_data(annotation_path, file_xml[0])

file name:  BloodImage_00000.xml
file name:  BloodImage_00001.xml
file name:  BloodImage_00002.xml
file name:  BloodImage_00003.xml
file name:  BloodImage_00004.xml
file name:  BloodImage_00005.xml
file name:  BloodImage_00006.xml
file name:  BloodImage_00007.xml
file name:  BloodImage_00008.xml
file name:  BloodImage_00009.xml
file name:  BloodImage_00010.xml
file name:  BloodImage_00011.xml
file name:  BloodImage_00012.xml
file name:  BloodImage_00013.xml
file name:  BloodImage_00014.xml
file name:  BloodImage_00015.xml
file name:  BloodImage_00016.xml
file name:  BloodImage_00017.xml
file name:  BloodImage_00018.xml
file name:  BloodImage_00019.xml
file name:  BloodImage_00020.xml
file name:  BloodImage_00021.xml
file name:  BloodImage_00022.xml
file name:  BloodImage_00023.xml
file name:  BloodImage_00024.xml
file name:  BloodImage_00026.xml
file name:  BloodImage_00028.xml
file name:  BloodImage_00029.xml
file name:  BloodImage_00030.xml
file name:  BloodImage_00031.xml
file name:

In [27]:
def copy_data(img_set_source, img_labels_root, imgs_source, type):
    """
    All imgs with annotation file converted into YOLO format will be seperate into train, test, val
    Function to seperate all data into 3 folder saved as their name folder respectively
    """

    file_name = img_set_source + '/' + type + ".txt"
    file = open(file_name)
    # Create folder for train imgs
    root_file = Path(data_root + des_imgs_path + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exist")
        os.makedirs(root_file)

    root_file = Path(data_root + des_labels_path + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exist")
        os.makedirs(root_file)
    # Read file to take file name
    for line in file.readlines():
        print(line)
        img_name = line.strip('\n')
        img_sor_file = imgs_source + '/' + img_name + '.jpg'          # Train img
        label_sor_file = img_labels_root + '/' + img_name + '.txt'    # Label train img

        # Copy image
        dict_dir = data_root + des_imgs_path + '/' + type
        img_dict_file = dict_dir + '/' + img_name + '.jpg'

        copyfile(img_sor_file, img_dict_file)

        # Copy label
        dict_dir = data_root + des_labels_path + '/' + type
        img_dict_file = dict_dir + '/' + img_name + '.txt'
        copyfile(label_sor_file, img_dict_file)

In [28]:
copy_data(imgset_root, label_root, img_path, "train")
copy_data(imgset_root, label_root, img_path, "val")
copy_data(imgset_root, label_root, img_path, "test")

Path images\train is not exist
BloodImage_00001

BloodImage_00003

BloodImage_00004

BloodImage_00005

BloodImage_00006

BloodImage_00008

BloodImage_00009

BloodImage_00010

BloodImage_00012

BloodImage_00013

BloodImage_00020

BloodImage_00022

BloodImage_00023

BloodImage_00024

BloodImage_00026

BloodImage_00032

BloodImage_00034

BloodImage_00036

BloodImage_00038

BloodImage_00039

BloodImage_00040

BloodImage_00042

BloodImage_00043

BloodImage_00044

BloodImage_00045

BloodImage_00046

BloodImage_00047

BloodImage_00048

BloodImage_00049

BloodImage_00050

BloodImage_00052

BloodImage_00054

BloodImage_00056

BloodImage_00059

BloodImage_00069

BloodImage_00070

BloodImage_00071

BloodImage_00076

BloodImage_00078

BloodImage_00079

BloodImage_00081

BloodImage_00082

BloodImage_00083

BloodImage_00086

BloodImage_00087

BloodImage_00090

BloodImage_00091

BloodImage_00092

BloodImage_00094

BloodImage_00095

BloodImage_00097

BloodImage_00100

BloodImage_00101

BloodImage_0010

In [29]:
print(len(os.listdir("images/train")))
print(len(os.listdir('images/val')))
print(len(os.listdir('images/test')))

205
87
72


In [32]:
!rm data/bccd.yaml
!echo "train: /content/drive/MyDrive/BloodcellDetection/BCCD/images/train" > data/bccd.yaml
!echo "val:   /content/drive/MyDrive/BloodcellDetection/BCCD/images/val" >> data/bccd.yaml
!echo 'is_coco: False'  >> data/bccd.yaml
!echo "nc : 3"    >> data/bccd.yaml
!echo "names: ['Platelets', 'RBC', 'WBC']" >> data/bccd.yaml
!cat data/bccd.yaml

'rm' is not recognized as an internal or external command,
operable program or batch file.
'cat' is not recognized as an internal or external command,
operable program or batch file.


Download pretrained weight for COCO dataset
https://github.com/meituan/YOLOv6/releases/download/0.1.0/yolov6s.pt

In [48]:
!mv /content/drive/MyDrive/BloodcellDetection/BCCD/YOLOv6/tools/train.py /content/drive/MyDrive/BloodcellDetection/BCCD/YOLOv6/train.py

'mv' is not recognized as an internal or external command,
operable program or batch file.


In [57]:
#Training
%cd d:
%cd ..
!python train.py --batch 32 --conf configs/yolov6s_finetune.py --data data/bccd.yaml --epochs 120

d:\yh\others\Project\Blood-Cell-Detection\YOLOv6\YOLOv6
d:\yh\others\Project\Blood-Cell-Detection\YOLOv6


Traceback (most recent call last):
  File "train.py", line 126, in <module>
    main(args)
  File "train.py", line 99, in main
    cfg, device, args = check_and_init(args)
  File "train.py", line 85, in check_and_init
    device = select_device(args.device)
  File "d:\yh\others\Project\Blood-Cell-Detection\YOLOv6\yolov6\utils\envs.py", line 32, in select_device
    assert torch.cuda.is_available()
AssertionError
